In [1]:
'''
https://github.com/Ishikawa7/Quick-paths-to-start/tree/main/Genetic%20algorithms
''';


In [1]:
import pandas as pd
import pygad
from math import floor


In [2]:
products_df = pd.read_csv('./products.csv')
products_df.head(20)


,Product,Space,Price,Quantity
0,Refrigerator A,0.751000,999.90,3
1,Cell phone,0.000009,2199.12,2
2,TV 55,0.400000,4346.99,5
3,TV 50,0.290000,3999.90,5
4,TV 42,0.200000,2999.00,5
5,Notebook A,0.003500,2499.90,4
6,Ventilator,0.496000,199.90,12
7,Microwave A,0.042400,308.66,7
8,Microwave B,0.054400,429.90,7
9,Microwave C,0.031900,299.29,9


In [10]:
max_volume = 5
num_generations = 10000
num_parents_mating = 4
mutation_percent_genes = 10
parent_selection_type = "sss"
crossover_type = "single_point"
mutation_type = "random"
keep_parents = 2
mutation_by_replacement = True
sol_per_pop = 40


In [11]:
#build the 'genome lists' but don't include more of a single item than can fit in max_volume
#essentially removing chromosomes that will by themselves result in exceeding the van volume
def build_genome_lists(products_df, max_volume):
    product_names = []
    product_volumes = []
    product_prices = []

    for row in products_df.itertuples():
        #include 'Quantity' of this value for easier list processing later
        #will use with a binary mask to determine products associated with a genome

        #do not include more of ITEM that can fit in max_volume
        max_qty_item = min(floor(max_volume/row.Space), row.Quantity)

        product_names = product_names + [row.Product]*max_qty_item
        product_volumes = product_volumes + [row.Space]*max_qty_item
        product_prices = product_prices + [row.Price]*max_qty_item

    return {'product_names':product_names,
            'product_volumes':product_volumes,
            'product_prices':product_prices}

#return values for the items in this knapsack (where item index == 1/True)
def get_van_items(solution, item_list):
    return [i for i in [tf and item for tf, item in zip(list(map(bool,solution)), item_list)] if i != False]

#fitness function, higher=better.  If volume > max_volume, return 0. else return value
max_value_found = 0
def fitness_func(ga_instance, solution, sol_idx):
    global max_volume, genome_lists, max_value_found

    sum_solution_volume = sum(get_van_items(solution, genome_lists.get('product_volumes')))

    #is the sum(item mass) in this knapsack < mass_limit
    if sum_solution_volume <= max_volume:
        #return sum(item value) in this van
        sum_solution_value = sum(get_van_items(solution, genome_lists.get('product_prices')))
        if sum_solution_value > max_value_found:
            max_value_found = sum_solution_value
            print(f'better solution found: (generation: {ga_instance.generations_completed} index: {sol_idx}): {sum_solution_volume} <= {max_volume} ::{sum_solution_value}')
        return sum_solution_value

    #sum(item mass) in this knapsack exceeds mass_limit, return 0
    return 0

'''
There appears to be a bug in PyGAD:

If no valid solution is found (fitness function returns 0 for all combinations),
PyGAD ga_instance.best_solution_generation will return 0 (the first genome) versus
-1 as in the documentation

To work around this IF ga_instance.best_solution_generation == 0 I must check the
genome volume and compare against max_volume
'''
def print_best_solution(ga_instance):

    #get the best solution
    best_solution_index = ga_instance.best_solution_generation
    print(f'best_solution_index: {best_solution_index}')

    if best_solution_index == -1:
        print("No solution found, increase generations")
    else:
        best_solution = ga_instance.best_solutions[ga_instance.best_solution_generation]
        if (best_solution_index == 0) and (round(sum(get_van_items(best_solution, genome_lists.get('product_volumes'))), 8) > max_volume):
            print('No solution found, increase generations')
            print('Bug use case - See notes above print_best_solution()')
        else:
            print(f'best_solution_generation: {ga_instance.best_solution_generation}')
            print(f'best_solution: {best_solution}')

            #Item names
            best_items_names = get_van_items(best_solution, genome_lists.get('product_names'))

            #calculate volumevalue
            best_items_volume = round(sum(get_van_items(best_solution, genome_lists.get('product_volumes'))), 8)
            sum_solution_value = round(sum(get_van_items(best_solution, genome_lists.get('product_prices'))), 2)

            print(f'items names: {best_items_names}')
            print(f'items volume: {best_items_volume}')
            print(f'items value: {sum_solution_value}')
            print(f'{sum_solution_value}/{best_items_volume}')

#run this instance
def run_instance():
    ga_instance = pygad.GA(num_generations=num_generations,
                           num_parents_mating=num_parents_mating,
                           fitness_func=fitness_func,
                           sol_per_pop=sol_per_pop,
                           num_genes=len(genome_lists['product_names']),
                           mutation_percent_genes=mutation_percent_genes,
                           parent_selection_type=parent_selection_type,
                           crossover_type=crossover_type,
                           mutation_type=mutation_type,
                           gene_type=int,
                           gene_space=[0,1],
                           save_best_solutions=True,
                           keep_parents=keep_parents,
                           mutation_by_replacement=mutation_by_replacement
                           )


    ga_instance.run()
    return ga_instance

#------------------------------------------------------------

genome_lists = build_genome_lists(products_df, max_volume)
print(f'max_genome_length/trimmed_genome_length: {sum(products_df.Quantity)}/{len(genome_lists["product_names"])}')

ga_instance = run_instance()
print_best_solution(ga_instance)


max_genome_length/trimmed_genome_length: 77/75
better solution found: (generation: 166 index: 19): 4.8609089899999995 <= 5 ::45516.910000000025
better solution found: (generation: 169 index: 32): 4.705408990000002 <= 5 ::50221.46000000003
better solution found: (generation: 171 index: 15): 4.851208989999999 <= 5 ::53873.700000000026
better solution found: (generation: 172 index: 25): 4.373408989999999 <= 5 ::54164.00000000002
better solution found: (generation: 172 index: 37): 4.700708990000002 <= 5 ::55724.60000000003
better solution found: (generation: 182 index: 18): 4.844708989999999 <= 5 ::58317.45000000002
better solution found: (generation: 188 index: 19): 4.9829 <= 5 ::60906.78000000003
better solution found: (generation: 202 index: 21): 4.846999999999999 <= 5 ::61704.780000000035
better solution found: (generation: 241 index: 17): 4.84081798 <= 5 ::63584.810000000034
better solution found: (generation: 262 index: 17): 4.930417980000001 <= 5 ::63824.94000000003
better solution 